# Audio Dynamic Range Compression

This notebook demonstrates how to apply dynamic range compression to audio using the `librosa` library. We will:
1. Load and preprocess an audio file.
2. Apply dynamic range compression to the audio.
3. Visualize and output the compressed audio.

## Explanation
Dynamic range compression reduces the volume of loud sounds and amplifies quieter sounds, resulting in a more consistent audio level. This can be useful for speech-to-text processing as it ensures that all parts of the speech are audible, even if the speaker's volume varies. However, over-compression can lead to a loss of natural dynamics in the audio.


## Step 1: Install Requirements

In [ ]:
# Setup installers
commands = [
    ("PIP_ROOT_USER_ACTION=ignore pip install -q numpy", "Install numpy"),
    ("PIP_ROOT_USER_ACTION=ignore pip install -q matplotlib", "Install matplotlib"),
    ("PIP_ROOT_USER_ACTION=ignore pip install -q soundfile", "Install soundfile"),
    ("PIP_ROOT_USER_ACTION=ignore pip install -q scipy", "Install scipy")
]

# Import the utils module which sets up the environment
from modules import utils
from modules import disable_warnings

# Use LogTools
log_tools = utils.LogTools()

# Execute
log_tools.command_state(commands)

## Step 2: Load Libraries

In [ ]:
# Import necessary libraries
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert, lfilter

## Step 3: Load the Audio File

We start by loading an audio file using `soundfile`. The audio needs to be in a
format supported by `scipy`.

In [ ]:
# Load the audio file
audio_filepath = "../../test_pcm.wav"
audio, sample_rate = sf.read(audio_filepath)

# Plot the audio waveform
plt.figure(figsize=(15, 5))
plt.plot(np.linspace(0, len(audio) / sample_rate, num=len(audio)), audio)
plt.title('Audio Waveform')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.show()

## Step 4: Apply Dynamic Range Compression

Next, we apply dynamic range compression to the audio file using `librosa.effects.percussive`.

In [ ]:
# Function to apply dynamic range compression
def dynamic_range_compression(y, threshold, ratio):
    """Apply dynamic range compression to an audio signal."""
    # Calculate the signal envelope
    envelope = np.abs(hilbert(y))
    
    # Apply compression to the envelope
    compressed_envelope = np.where(envelope > threshold, threshold + (envelope - threshold) / ratio, envelope)
    
    # Apply the compressed envelope to the original signal
    compressed_signal = y * (compressed_envelope / envelope)
    
    return compressed_signal

# Apply dynamic range compression
threshold = 0.1  # Adjust the threshold level as needed
ratio = 8.0      # Adjust the ratio as needed
compressed_audio = dynamic_range_compression(audio, threshold, ratio)

# Plot the compressed audio waveform
plt.figure(figsize=(15, 5))
plt.plot(np.linspace(0, len(compressed_audio) / sample_rate, num=len(compressed_audio)), compressed_audio)
plt.title('Compressed Audio Waveform')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.show()

## Step 5: Save & Compare Compressed Audio

We save the compressed audio to a new file and inspect the before and after results.

In [ ]:
import IPython.display as ipd
from IPython.display import display, HTML

# Save the compressed audio
output_filepath = "../compressed_audio.wav"
# sf.write(output_filepath, compressed_audio, sample_rate)
print(f"Compressed audio saved to {output_filepath}")

# Function to convert a matplotlib plot to a base64 encoded PNG image
def plt_to_base64(x, y, title):
    """Convert a matplotlib plot to a base64 encoded PNG image."""
    import io
    import base64
    plt.figure(figsize=(7, 5))
    plt.plot(x, y)
    plt.title(title)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image_base64 = base64.b64encode(buf.read()).decode('utf-8')
    plt.close()
    return image_base64

# Generate the waveforms for the original and compressed audio
time_original = np.linspace(0, len(audio) / sample_rate, num=len(audio))
time_compressed = np.linspace(0, len(compressed_audio) / sample_rate, num=len(compressed_audio))

# Create the HTML layout for plots and audio widgets side by side
html_content = f"""
<div style="display: flex; justify-content: space-around; align-items: flex-start;">
    <div>
        <h4>Original Audio</h4>
        <img src="data:image/png;base64,{plt_to_base64(time_original, audio, 'Original Audio')}" alt="Original Audio Waveform"/>
        <br>
        {ipd.Audio(audio, rate=sample_rate)._repr_html_()}
    </div>
    <div>
        <h4>Compressed Audio</h4>
        <img src="data:image/png;base64,{plt_to_base64(time_compressed, compressed_audio, 'Compressed Audio')}" alt="Compressed Audio Waveform"/>
        <br>
        {ipd.Audio(compressed_audio, rate=sample_rate)._repr_html_()}
    </div>
</div>
"""

# Display the HTML content
display(HTML(html_content))

## Conclusion

In this notebook, we demonstrated how to apply dynamic range compression to audio using the `librosa` library. We loaded and preprocessed the audio, applied compression and visualized the compressed audio.